<a href="https://colab.research.google.com/github/Xiaoyuu99/CACC/blob/main/CCAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


In [ ]:
train = pd.read_csv("DIWBB_Training.csv")
test = pd.read_csv("DIWBB_Test.csv")

<ipython-input-20-e9c63138266f>:1: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("DIWBB_Training.csv")


In [ ]:
print(train.shape)
print(train.info())
print(train.head())

print(test.shape)
print(test.info())
print(test.head())

(209266, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209266 entries, 0 to 209265
Data columns (total 25 columns):
 #   Column                                          Non-Null Count   Dtype 
---  ------                                          --------------   ----- 
 0   RecordID                                        209266 non-null  int64 
 1   ChampionshipYear                                209266 non-null  int64 
 2   CustomerID                                      209266 non-null  int64 
 3   CustomerCity                                    209266 non-null  object
 4   CustomerState                                   208668 non-null  object
 5   CustomerZipCode                                 209266 non-null  object
 6   CustomerInstitutionAffinity                     209266 non-null  object
 7   IsCustomerInNCAAMembership                      209266 non-null  object
 8   HasCustomerClickedOrOpenedEmailsSixMonthsPrior  209266 non-null  object
 9   CustomerFirstWBBActionDa

In [ ]:
# Function to convert date string to standard format
def convert_to_standard_format(date_str):
    try:
        return datetime.strptime(date_str, '%m/%d/%y').date()
    except (ValueError, TypeError):
        return pd.NA  # Return pd.NA if conversion fails

# Convert date strings to datetime objects
train['standard_date_WBBA_First'] = train['CustomerFirstWBBActionDate'].apply(convert_to_standard_format)
train['standard_date_WBBA_Last'] = train['CustomerLastWBBActionDate'].apply(convert_to_standard_format)
train['standard_date_WBBP_First'] = train['CustomerFirstWBBPurchaseDate'].apply(convert_to_standard_format)
train['standard_date_WBBP_Last'] = train['CustomerLastWBBPurchaseDate'].apply(convert_to_standard_format)

test['standard_date_WBBA_First'] = test['CustomerFirstWBBActionDate'].apply(convert_to_standard_format)
test['standard_date_WBBA_Last'] = test['CustomerLastWBBActionDate'].apply(convert_to_standard_format)
test['standard_date_WBBP_First'] = test['CustomerFirstWBBPurchaseDate'].apply(convert_to_standard_format)
test['standard_date_WBBP_Last'] = test['CustomerLastWBBPurchaseDate'].apply(convert_to_standard_format)


# Convert NaN values to float 0.0
train['standard_date_WBBA_First'] = train['standard_date_WBBA_First'].fillna(0.0)
train['standard_date_WBBA_Last'] = train['standard_date_WBBA_Last'].fillna(0.0)
train['standard_date_WBBP_First'] = train['standard_date_WBBP_First'].fillna(0.0)
train['standard_date_WBBP_Last'] = train['standard_date_WBBP_Last'].fillna(0.0)

test['standard_date_WBBA_First'] = test['standard_date_WBBA_First'].fillna(0.0)
test['standard_date_WBBA_Last'] = test['standard_date_WBBA_Last'].fillna(0.0)
test['standard_date_WBBP_First'] = test['standard_date_WBBP_First'].fillna(0.0)
test['standard_date_WBBP_Last'] = test['standard_date_WBBP_Last'].fillna(0.0)

# Calculate duration for valid dates
train["duration_WBBA"] = train['standard_date_WBBA_Last'] - train['standard_date_WBBA_First']
train["duration_WBBP"] = train['standard_date_WBBP_Last'] - train['standard_date_WBBP_First']

test["duration_WBBA"] = test['standard_date_WBBA_Last'] - test['standard_date_WBBA_First']
test["duration_WBBP"] = test['standard_date_WBBP_Last'] - test['standard_date_WBBP_First']

print(train[["duration_WBBA","duration_WBBP"]])

print(test[["duration_WBBA","duration_WBBP"]])

       duration_WBBA duration_WBBP
0            0:00:00       0:00:00
1            0:00:00           0.0
2            0:00:00       0:00:00
3            0:00:00       0:00:00
4            0:00:00       0:00:00
...              ...           ...
209261       0:00:00           0.0
209262       0:00:00           0.0
209263       0:00:00           0.0
209264       0:00:00       0:00:00
209265       0:00:00       0:00:00

[209266 rows x 2 columns]
            duration_WBBA       duration_WBBP
0                 0:00:00                 0.0
1                 0:00:00             0:00:00
2                 0:00:00             0:00:00
3      1583 days, 0:00:00  1583 days, 0:00:00
4                 0:00:00             0:00:00
...                   ...                 ...
20930                 0.0                 0.0
20931             0:00:00                 0.0
20932                 0.0                 0.0
20933                 0.0                 0.0
20934                 0.0                 0.0



In [ ]:
# Assuming 'train_pre' is your DataFrame containing the data
def get_active_indicator(duration):
    if duration == 0.0 or 'days' in str(duration):
        return 1
    else:
        return 0

# Add a new column based on the conditions
train['Active_indicator_WBBA'] = train['duration_WBBA'].apply(get_active_indicator)
test['Active_indicator_WBBA'] = test['duration_WBBA'].apply(get_active_indicator)

train['Active_indicator_WBBP'] = train['duration_WBBP'].apply(get_active_indicator)
test['Active_indicator_WBBP'] = test['duration_WBBP'].apply(get_active_indicator)
# Print the DataFrame to verify the changes
print(train)
print(test)

        RecordID  ChampionshipYear  CustomerID CustomerCity CustomerState  \
0              4              2022      757758   Huntsville            TX   
1             14              2022      574752        Salem            OR   
2             20              2022      741837        (UNK)           UNK   
3             21              2022      237618        (UNK)           UNK   
4             23              2022      799761        (UNK)           UNK   
...          ...               ...         ...          ...           ...   
209261   1594423              2023      867155   Pine Bluff            AR   
209262   1594427              2023      662341   Washington            WA   
209263   1594428              2023       38418        (UNK)           UNK   
209264   1594445              2023       74611        (UNK)           UNK   
209265   1594446              2023      318964        (UNK)           UNK   

       CustomerZipCode CustomerInstitutionAffinity IsCustomerInNCAAMembersh

In [ ]:
train['duration_WBBA']

0                   0:00:00
1                   0:00:00
2                   0:00:00
3                   0:00:00
4                   0:00:00
                ...        
83034               0:00:00
83035    1784 days, 0:00:00
83036               0:00:00
83037               0:00:00
83038                   0.0
Name: duration_WBBA, Length: 83039, dtype: object

In [ ]:
# Define the function to extract days
def extract_days(duration_timedelta):
    # Convert timedelta object to string
    duration_str = str(duration_timedelta)

    # Check if the duration string contains days
    if 'days' in duration_str:
        # Split the duration into days and time components
        days, _ = duration_str.split(', ')
        # Extract the number of days
        num_days = int(days.split()[0])
    else:
        num_days = 0
    return num_days

# Apply the function to the 'duration_WBBA' column
train['duration_WBBA'] = train['duration_WBBA'].apply(extract_days)
test['duration_WBBA'] = test['duration_WBBA'].apply(extract_days)

train['duration_WBBP'] = train['duration_WBBP'].apply(extract_days)
test['duration_WBBP'] = test['duration_WBBP'].apply(extract_days)
# Print the DataFrame to verify the changes
print(test['duration_WBBP'])


0           0
1           0
2           0
3        1583
4           0
         ... 
20930       0
20931       0
20932       0
20933       0
20934       0
Name: duration_WBBP, Length: 20935, dtype: int64


In [ ]:
train['duration_WBBA'] = pd.to_numeric(train['duration_WBBA'], errors='coerce')
train['duration_WBBP'] = pd.to_numeric(train['duration_WBBP'], errors='coerce')

test['duration_WBBA'] = pd.to_numeric(test['duration_WBBA'], errors='coerce')
test['duration_WBBP'] = pd.to_numeric(test['duration_WBBP'], errors='coerce')

print(test['duration_WBBP'])

0           0
1           0
2           0
3        1583
4           0
         ... 
20930       0
20931       0
20932       0
20933       0
20934       0
Name: duration_WBBP, Length: 20935, dtype: int64


In [ ]:
# Fill NULL values under specified columns with 'NoActivity'
columns_to_fill = ['EventRoundName', 'IsEventFinalSite', 'EventSession', 'HostingInstitution', 'FacilityState', 'FacilityZipCode']
train[columns_to_fill] = train[columns_to_fill].fillna('NoActivity')
test[columns_to_fill] = test[columns_to_fill].fillna('NoActivity')

In [ ]:
# Assuming train is your DataFrame
train['event_date_indicator'] = train['EventBeginDate'].notnull().astype(int)
test['event_date_indicator'] = test['EventBeginDate'].notnull().astype(int)

In [ ]:
train_pre = train[["ChampionshipYear", "CustomerID", "IsCustomerInNCAAMembership", "HasCustomerClickedOrOpenedEmailsSixMonthsPrior", "duration_WBBA", "duration_WBBP", 'EventRoundName', 'IsEventFinalSite', 'EventSession', 'HostingInstitution', 'FacilityState', 'FacilityZipCode', 'event_date_indicator','Active_indicator_WBBP','Active_indicator_WBBA', 'ActivityType']]
test_pre = test[["ChampionshipYear", "CustomerID", "IsCustomerInNCAAMembership", "HasCustomerClickedOrOpenedEmailsSixMonthsPrior", "duration_WBBA", "duration_WBBP", 'EventRoundName', 'IsEventFinalSite', 'EventSession', 'HostingInstitution', 'FacilityState', 'FacilityZipCode', 'event_date_indicator','Active_indicator_WBBP','Active_indicator_WBBA']]
print(train_pre.head())
print(test_pre.head())

   ChampionshipYear  CustomerID IsCustomerInNCAAMembership  \
0              2022      757758                         No   
1              2022      574752                         No   
2              2022      741837                         No   
3              2022      237618                         No   
4              2022      799761                         No   

  HasCustomerClickedOrOpenedEmailsSixMonthsPrior  duration_WBBA  \
0                                          (N/A)              0   
1                                            Yes              0   
2                                            Yes              0   
3                                            Yes              0   
4                                          (N/A)              0   

   duration_WBBP EventRoundName IsEventFinalSite EventSession  \
0              0     NoActivity       NoActivity   NoActivity   
1              0     NoActivity       NoActivity   NoActivity   
2              0     NoActivi

In [ ]:
# Convert 'ChampionshipYear' column to string
test_pre['ChampionshipYear'] = test_pre['ChampionshipYear'].astype(int)

train_pre['ChampionshipYear'] = train_pre['ChampionshipYear'].astype(int)


<ipython-input-29-bbae8cb670f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pre['ChampionshipYear'] = test_pre['ChampionshipYear'].astype(int)
<ipython-input-29-bbae8cb670f3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre['ChampionshipYear'] = train_pre['ChampionshipYear'].astype(int)


In [ ]:
print(test_pre.head())
print(train_pre.head())

   ChampionshipYear  CustomerID IsCustomerInNCAAMembership  \
0              2022      552730                         No   
1              2022       84338                         No   
2              2022      656065                         No   
3              2022      419848                         No   
4              2022      338825                         No   

  HasCustomerClickedOrOpenedEmailsSixMonthsPrior  duration_WBBA  \
0                                          (N/A)              0   
1                                            Yes              0   
2                                          (N/A)              0   
3                                            Yes           1583   
4                                          (N/A)              0   

   duration_WBBP EventRoundName IsEventFinalSite EventSession  \
0              0     NoActivity       NoActivity   NoActivity   
1              0     NoActivity       NoActivity   NoActivity   
2              0     NoActivi

In [ ]:
# Define the mapping dictionary
activity_mapping = {
    'No Activity': 0,
    'Primary Purchase': 1,
    'Secondary Purchase': 2,
    'Transfer Recipient': 3,
    'Multiple Activities': 4,
    'Other Secondary Activity': 5
}

# Replace values in the 'ActivityType' column using the mapping dictionary
train_pre['ActivityType'] = train_pre['ActivityType'].replace(activity_mapping)

# Display the updated DataFrame
print(train_pre)


        ChampionshipYear  CustomerID IsCustomerInNCAAMembership  \
0                   2022      757758                         No   
1                   2022      574752                         No   
2                   2022      741837                         No   
3                   2022      237618                         No   
4                   2022      799761                         No   
...                  ...         ...                        ...   
209261              2023      867155                         No   
209262              2023      662341                         No   
209263              2023       38418                         No   
209264              2023       74611                         No   
209265              2023      318964                         No   

       HasCustomerClickedOrOpenedEmailsSixMonthsPrior  duration_WBBA  \
0                                               (N/A)              0   
1                                                 Y

<ipython-input-30-d4d4d5213b8c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre['ActivityType'] = train_pre['ActivityType'].replace(activity_mapping)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import pickle

# Assuming train_pre is your training data
cat_vars = ['ChampionshipYear', 'CustomerID', 'IsCustomerInNCAAMembership',
            'HasCustomerClickedOrOpenedEmailsSixMonthsPrior', 'EventRoundName',
            'IsEventFinalSite', 'EventSession', 'HostingInstitution',
            'FacilityState', 'FacilityZipCode', 'event_date_indicator']

# Initialize dictionaries to store encoders
label_encoders = {}

# Apply label encoding to categorical variables
for var in cat_vars:
    encoder = LabelEncoder()
    train_pre[var] = encoder.fit_transform(train_pre[var])
    label_encoders[var] = encoder

# Split data into features and target
X = train_pre.drop(columns=['ActivityType'])  # Features
y = train_pre['ActivityType']  # Target

# Encode target variable
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Train XGBoost model
params = {
    'objective': 'multi:softmax',  # Specify multi-class classification
    'num_class': len(np.unique(y)),  # Number of classes
    'eval_metric': 'merror'  # Evaluation metric
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
num_round = 100
bst = xgb.train(params, dtrain, num_round, evals=[(dtrain, 'train'), (dval, 'val')])

# Predict on validation set
y_pred = bst.predict(dval)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

# Save the trained model and encoders
bst.save_model('xgboost_model.model')
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

print("Model and encoders saved successfully.")


<ipython-input-16-fa835d46da20>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre[var] = encoder.fit_transform(train_pre[var])
<ipython-input-16-fa835d46da20>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre[var] = encoder.fit_transform(train_pre[var])
<ipython-input-16-fa835d46da20>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

[0]	train-merror:0.00002	val-merror:0.00000
[1]	train-merror:0.00002	val-merror:0.00000
[2]	train-merror:0.00002	val-merror:0.00000
[3]	train-merror:0.00002	val-merror:0.00000
[4]	train-merror:0.00002	val-merror:0.00000
[5]	train-merror:0.00002	val-merror:0.00000
[6]	train-merror:0.00002	val-merror:0.00000
[7]	train-merror:0.00002	val-merror:0.00000
[8]	train-merror:0.00002	val-merror:0.00000
[9]	train-merror:0.00002	val-merror:0.00000
[10]	train-merror:0.00002	val-merror:0.00000
[11]	train-merror:0.00002	val-merror:0.00000
[12]	train-merror:0.00002	val-merror:0.00000
[13]	train-merror:0.00002	val-merror:0.00000
[14]	train-merror:0.00002	val-merror:0.00000
[15]	train-merror:0.00002	val-merror:0.00000
[16]	train-merror:0.00002	val-merror:0.00000
[17]	train-merror:0.00002	val-merror:0.00000
[18]	train-merror:0.00002	val-merror:0.00000
[19]	train-merror:0.00002	val-merror:0.00000
[20]	train-merror:0.00002	val-merror:0.00000
[21]	train-merror:0.00002	val-merror:0.00000
[22]	train-merror:0.

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:41:03] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import pickle

# Assuming train_pre is your training data
num_vars = ['duration_WBBA', 'duration_WBBP', 'event_date_indicator']
cat_vars = ['IsCustomerInNCAAMembership', 'CustomerID',
            'HasCustomerClickedOrOpenedEmailsSixMonthsPrior', 'EventRoundName',
            'IsEventFinalSite', 'EventSession', 'HostingInstitution',
            'FacilityState', 'FacilityZipCode']

# Initialize dictionaries to store encoders
label_encoders = {}

# Apply label encoding to categorical variables, excluding specified variables
for var in cat_vars:
    encoder = LabelEncoder()
    train_pre[var] = encoder.fit_transform(train_pre[var])
    label_encoders[var] = encoder

# Split data into features and target
X_cat = train_pre[cat_vars]  # Categorical features
X_num = train_pre[num_vars]  # Numerical features
X = pd.concat([X_cat, X_num], axis=1)  # Features

# Encode target variable
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_pre['ActivityType'])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size=0.3, random_state=42)

# Train XGBoost model
params = {
    'objective': 'multi:softmax',  # Specify multi-class classification
    'num_class': len(np.unique(y_train)),  # Number of classes
    'eval_metric': 'merror'  # Evaluation metric
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
num_round = 100
bst = xgb.train(params, dtrain, num_round, evals=[(dtrain, 'train'), (dval, 'val')])

# Predict on validation set
y_pred = bst.predict(dval)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

# Save the trained model and encoders
bst.save_model('xgboost_model.model')
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)



[0]	train-merror:0.00002	val-merror:0.00000
[1]	train-merror:0.00002	val-merror:0.00000


<ipython-input-17-da1b78114b06>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre[var] = encoder.fit_transform(train_pre[var])
<ipython-input-17-da1b78114b06>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre[var] = encoder.fit_transform(train_pre[var])
<ipython-input-17-da1b78114b06>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

[2]	train-merror:0.00002	val-merror:0.00000
[3]	train-merror:0.00002	val-merror:0.00000
[4]	train-merror:0.00002	val-merror:0.00000
[5]	train-merror:0.00002	val-merror:0.00000
[6]	train-merror:0.00002	val-merror:0.00000
[7]	train-merror:0.00002	val-merror:0.00000
[8]	train-merror:0.00002	val-merror:0.00000
[9]	train-merror:0.00002	val-merror:0.00000
[10]	train-merror:0.00002	val-merror:0.00000
[11]	train-merror:0.00002	val-merror:0.00000
[12]	train-merror:0.00002	val-merror:0.00000
[13]	train-merror:0.00002	val-merror:0.00000
[14]	train-merror:0.00002	val-merror:0.00000
[15]	train-merror:0.00002	val-merror:0.00000
[16]	train-merror:0.00002	val-merror:0.00000
[17]	train-merror:0.00002	val-merror:0.00000
[18]	train-merror:0.00002	val-merror:0.00000
[19]	train-merror:0.00002	val-merror:0.00000
[20]	train-merror:0.00002	val-merror:0.00000
[21]	train-merror:0.00002	val-merror:0.00000
[22]	train-merror:0.00002	val-merror:0.00000
[23]	train-merror:0.00002	val-merror:0.00000
[24]	train-merror:

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:41:05] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import pickle

# Load the trained XGBoost model
bst = xgb.Booster()
bst.load_model('xgboost_model.model')

# Load the saved label encoders
with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Assume test_pre is your test data

# Prepare the test data
test_cat = test_pre[cat_vars]  # Categorical features
test_num = test_pre[num_vars]  # Numerical features

# Apply label encoding using the loaded encoders
for var, encoder in label_encoders.items():
    if var in cat_vars:
        # Transform, not fit_transform
        test_cat[var] = encoder.transform(test_cat[var])

# Concatenate categorical and numerical features
test_data = pd.concat([test_cat, test_num], axis=1)

# Convert test data to DMatrix
dtest = xgb.DMatrix(test_data)

# Make predictions on the test dataset
y_pred = bst.predict(dtest)

# Assuming you have the label encoder object for the target variable
# If 'ActivityType' is the target variable
encoder = label_encoders['ActivityType']

# Reverse encoding on predictions to convert them back to original content
y_pred_original = encoder.inverse_transform(y_pred.astype(int))

# Now you have the predicted 'ActivityType' in the original format
print(y_pred_original)



ValueError: invalid literal for int() with base 10: 'No'

In [ ]:
print(test_pre)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import pickle
import pandas as pd

# Assuming train_pre is your training data
num_vars = ['duration_WBBA', 'duration_WBBP', 'event_date_indicator']
cat_vars = ['IsCustomerInNCAAMembership', 'CustomerID',
            'HasCustomerClickedOrOpenedEmailsSixMonthsPrior', 'EventRoundName',
            'IsEventFinalSite', 'EventSession', 'HostingInstitution',
            'FacilityState', 'FacilityZipCode']

# Initialize dictionaries to store encoders
label_encoders = {}

# Apply label encoding to categorical variables, excluding specified variables
for var in cat_vars:
    encoder = LabelEncoder()
    train_pre[var] = encoder.fit_transform(train_pre[var])
    label_encoders[var] = encoder

# Split data into features and target
X_cat = train_pre[cat_vars]  # Categorical features
X_num = train_pre[num_vars]  # Numerical features
X = pd.concat([X_cat, X_num], axis=1)  # Features

# Target variable
y_train = train_pre['ActivityType']  # No label encoding

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size=0.3, random_state=42)

# Train XGBoost model
params = {
    'objective': 'multi:softmax',  # Specify multi-class classification
    'num_class': len(np.unique(y_train)),  # Number of classes
    'eval_metric': 'merror'  # Evaluation metric
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
num_round = 100
bst = xgb.train(params, dtrain, num_round, evals=[(dtrain, 'train'), (dval, 'val')])

# Predict on validation set
y_pred = bst.predict(dval)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

# Save the trained model and encoders
bst.save_model('xgboost_model.model')
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)


<ipython-input-31-637bd29696e3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre[var] = encoder.fit_transform(train_pre[var])
<ipython-input-31-637bd29696e3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pre[var] = encoder.fit_transform(train_pre[var])
<ipython-input-31-637bd29696e3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

[0]	train-merror:0.01735	val-merror:0.01765
[1]	train-merror:0.01739	val-merror:0.01766
[2]	train-merror:0.01737	val-merror:0.01765
[3]	train-merror:0.01736	val-merror:0.01771
[4]	train-merror:0.01729	val-merror:0.01770
[5]	train-merror:0.01733	val-merror:0.01765
[6]	train-merror:0.01731	val-merror:0.01768
[7]	train-merror:0.01731	val-merror:0.01768
[8]	train-merror:0.01724	val-merror:0.01768
[9]	train-merror:0.01721	val-merror:0.01768
[10]	train-merror:0.01716	val-merror:0.01773
[11]	train-merror:0.01707	val-merror:0.01765
[12]	train-merror:0.01705	val-merror:0.01760
[13]	train-merror:0.01701	val-merror:0.01759
[14]	train-merror:0.01701	val-merror:0.01760
[15]	train-merror:0.01696	val-merror:0.01754
[16]	train-merror:0.01690	val-merror:0.01755
[17]	train-merror:0.01688	val-merror:0.01759
[18]	train-merror:0.01682	val-merror:0.01760
[19]	train-merror:0.01682	val-merror:0.01755
[20]	train-merror:0.01678	val-merror:0.01755
[21]	train-merror:0.01674	val-merror:0.01749
[22]	train-merror:0.

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:42:30] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import pickle

# Load the trained XGBoost model
bst = xgb.Booster()
bst.load_model('xgboost_model.model')

# Load the saved label encoders
with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Prepare the test data
test_cat = test_pre[cat_vars].copy()  # Categorical features
test_num = test_pre[num_vars].copy()  # Numerical features

# Apply label encoding using the loaded encoders
for var in cat_vars:
    encoder = LabelEncoder()
    test_pre[var] = encoder.fit_transform(test_pre[var])
    label_encoders[var] = encoder

# Concatenate categorical and numerical features
test_data = pd.concat([test_cat, test_num], axis=1)

# Convert test data to DMatrix
dtest = xgb.DMatrix(test_data, enable_categorical = True)

# Make predictions on the test dataset
y_pred = bst.predict(dtest)

# Now you have the predicted 'ActivityType' directly
print(y_pred)


<ipython-input-34-e139d1f7aaca>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pre[var] = encoder.fit_transform(test_pre[var])
<ipython-input-34-e139d1f7aaca>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pre[var] = encoder.fit_transform(test_pre[var])
<ipython-input-34-e139d1f7aaca>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[0. 0. 0. ... 1. 1. 1.]


In [ ]:
# Define the mapping dictionary
activity_mapping = {
    0: 'No Activity',
    1: 'Primary Purchase',
    2: 'Secondary Purchase',
    3: 'Transfer Recipient',
    4: 'Multiple Activities',
    5: 'Other Secondary Activity'
}

# Replace numbers in y_pred with activity labels
y_pred_labels = [activity_mapping[pred] for pred in y_pred]

# Now you have the predicted 'ActivityType' labels
print(y_pred_labels)

# Create a DataFrame from the list
df = pd.DataFrame({'ActivityType': y_pred_labels})

# Save the DataFrame to a CSV file
df.to_csv('predicted_activity_labels.csv', index=False)


['No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activity', 'No Activ

In [ ]:
# Reverse encoding of feature names
decoded_importance = {}
for feature, score in importance.items():
    if feature in label_encoders:
        decoded_feature = label_encoders[feature].inverse_transform([int(feature)])[0]
    else:
        decoded_feature = feature
    decoded_importance[decoded_feature] = score

# Print decoded feature importances
print("Decoded Feature Importances:")
for feature, score in decoded_importance.items():
    print(f"{feature}: {score}")

# If you want to see the decoded feature importances in a sorted manner
decoded_sorted_importance = sorted(decoded_importance.items(), key=lambda x: x[1], reverse=True)
print("\nSorted Decoded Feature Importances:")
for feature, score in decoded_sorted_importance:
    print(f"{feature}: {score}")





Decoded Feature Importances:
f0: 104.0
f1: 5709.0
f2: 901.0
f3: 315.0
f4: 35.0
f5: 1545.0
f6: 611.0
f7: 496.0
f8: 538.0
f9: 725.0
f10: 395.0

Sorted Decoded Feature Importances:
f1: 5709.0
f5: 1545.0
f2: 901.0
f9: 725.0
f6: 611.0
f8: 538.0
f7: 496.0
f10: 395.0
f3: 315.0
f0: 104.0
f4: 35.0


In [ ]:
print(f"{feature}")

f4
